In [1]:
#pip install selenium

In [1]:
import json
import pandas as pd
import gzip
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances
from bs4 import BeautifulSoup

from requests import get
import requests

from selenium import webdriver
import time

from random import randint
from time import sleep

In [2]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield eval(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Clothing_Shoes_and_Jewelry_5.json.gz')

In [3]:
df.head(60)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A1KLRMWW2FWPL4,0000031887,"Amazon Customer ""cameramom""","[0, 0]",This is a great tutu and at a really great pri...,5.0,Great tutu- not cheaply made,1297468800,"02 12, 2011"
1,A2G5TCU2WDFZ65,0000031887,Amazon Customer,"[0, 0]",I bought this for my 4 yr old daughter for dan...,5.0,Very Cute!!,1358553600,"01 19, 2013"
2,A1RLQXYNCMWRWN,0000031887,Carola,"[0, 0]",What can I say... my daughters have it in oran...,5.0,I have buy more than one,1357257600,"01 4, 2013"
3,A8U3FAMSJVHS5,0000031887,Caromcg,"[0, 0]","We bought several tutus at once, and they are ...",5.0,"Adorable, Sturdy",1398556800,"04 27, 2014"
4,A3GEOILWLK86XM,0000031887,CJ,"[0, 0]",Thank you Halo Heaven great product for Little...,5.0,Grammy's Angels Love it,1394841600,"03 15, 2014"
5,A27UF1MSF3DB2,0000031887,"C-Lo ""Cynthia""","[0, 0]",I received this today and I'm not a fan of it ...,4.0,It's ok,1396224000,"03 31, 2014"
6,A16GFPNVF4Y816,0000031887,design maven,"[0, 0]",Bought this as a backup to the regular ballet ...,5.0,Great for dress-up and for ballet practice,1399075200,"05 3, 2014"
7,A2M2APVYIB2U6K,0000031887,Jamie P.,"[0, 0]",Great tutu for a great price. It isn't a &#34;...,5.0,Great value,1356220800,"12 23, 2012"
8,A1NJ71X3YPQNQ9,0000031887,JBerger,"[0, 0]","My daughter liked this, and it with her costum...",4.0,Good,1384041600,"11 10, 2013"
9,A3EERSWHAI6SO,0000031887,"Jeffrey Hollingshead ""Jillian hollingshead""","[7, 8]",For what I paid for two tutus is unbeatable an...,5.0,WOW !! ..is all I have to say!,1349568000,"10 7, 2012"


In [4]:
#Check size of dataset 
df.shape

(278677, 9)

In [5]:
df.columns

Index(['reviewerID', 'asin', 'reviewerName', 'helpful', 'reviewText',
       'overall', 'summary', 'unixReviewTime', 'reviewTime'],
      dtype='object')

In [6]:
df[df['asin'] == 'B00971NLBS']

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
224886,A1J4GHI1VRLCX9,B00971NLBS,Alya,"[1, 1]",I will be purchasing more of this brand of shi...,5.0,Classic,1374624000,"07 24, 2013"
224887,A21UXFFQGQS4YJ,B00971NLBS,"Anne Harris ""Lady H""","[0, 0]",I found it to be a little roomy but it will be...,5.0,nice dress shirt,1393545600,"02 28, 2014"
224888,A22Z5LSEIPN8DU,B00971NLBS,Ashleyy,"[0, 0]","I ordered this shirt in a light blue, size two...",2.0,"Wrong color, wrong size.",1372291200,"06 27, 2013"
224889,A3SP43WB89B2UE,B00971NLBS,"cecilafayette ""ceci""","[0, 0]","This shirt fits well, looks nice, and is a gre...",4.0,Great Little Basic Shirt,1388102400,"12 27, 2013"
224890,A1Q3MDPIS017JE,B00971NLBS,Kay Z,"[0, 0]","The material is a little thin, but that was ex...",4.0,"Good fit, little thin",1400284800,"05 17, 2014"
224891,A1BCJXOINBPB38,B00971NLBS,"lascrucesgirl ""Growing boy's momma""","[3, 4]",I did not use the tie for my son. It was dress...,5.0,Nicely made and looks sharp.,1360627200,"02 12, 2013"
224892,A1AFORE8L2LGGW,B00971NLBS,mandamommyof3,"[5, 5]",This is adorable! I would buy more if the tie...,4.0,Tie is too short,1356652800,"12 28, 2012"
224893,A22E4B018G6NX7,B00971NLBS,RDeering,"[0, 0]",Nice little shirt I couldn't find elsewhere th...,5.0,can't complain,1388966400,"01 6, 2014"


In [7]:
#Check how many unique items are in the data 
len(df['asin'].unique())

23033

In [8]:
#Created a non-repeating list of product ID #s with the intent of looping through them in the url to scrape. 
asin_set = set(df['asin'])
asin_set = list(asin_set)

In [10]:
asin_set.sort()

In [11]:
asin_set

['0000031887',
 '0123456479',
 '1608299953',
 '1617160377',
 'B00001W0KA',
 'B00001WRHJ',
 'B00004SR8W',
 'B00004SR8Z',
 'B00004SR9P',
 'B00004U1J2',
 'B000051SEN',
 'B000051SEP',
 'B00005JHKE',
 'B00005JSBK',
 'B00005KJXN',
 'B00005TQI7',
 'B0000643Q8',
 'B000067R84',
 'B00006I551',
 'B00006XXGO',
 'B0000722HK',
 'B000072UMJ',
 'B000074RL3',
 'B000078CYM',
 'B00007AS82',
 'B00007FFL9',
 'B00007GD8X',
 'B00007GD9I',
 'B00007GD9W',
 'B00007GDAL',
 'B00007GDD3',
 'B00007GDG5',
 'B00007IVVR',
 'B000086211',
 'B0000862FI',
 'B0000864CZ',
 'B0000865II',
 'B0000866JI',
 'B000086778',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B0000868O9',
 'B000086910',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B0000891IO',
 'B0000891K0',
 'B00008AALU',
 'B00008ECKG',
 'B00008I8YM',
 'B00008ID0B',
 'B00008ID0L',
 'B00008ID1O',
 'B00008ID39',
 'B00008IEUW',
 'B00008IOHR',
 'B00008IPCG',
 'B00008KH9C',
 'B00008KI3U',
 'B00008LUS5',
 'B00009ESZI',
 'B00009OXE8',
 'B00009QB5V',
 'B00009QMR3',
 'B00009R8

In [12]:
#Check how many unique users are in the data. 
len(df['reviewerID'].unique())

39387

EDA

Scrape additional data using Selenium

In [25]:
driver = webdriver.Chrome(executable_path="../chromedriver/macos/chromedriver") 

In [18]:
soup = BeautifulSoup(driver.page_source, 'lxml')

In [24]:
t1 = time.time()
print(t1)
list_of_dicts = []
for i in redo_list2:
    print(f'https://www.amazon.com/gp/product/{i}')
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['name'] = soup.find('span', {'id':'productTitle'}).text.strip()

        product_info['description'] = soup.find('ul', {'class':'a-unordered-list a-vertical a-spacing-none'}).text.strip()
        
        product_info['category'] = soup.find('ul', {'class': 'a-unordered-list a-horizontal a-size-small'}).text.strip()
        
        product_info['size'] = soup.find('div', {'id': 'variation_size_name'})
        
        product_info['details'] = soup.find_all('td', {'class': 'a-span7 a-size-base'})
        
        if soup.find('div', {'id': 'variation_color_name'}):
            product_info['color'] = soup.find('div', {'id': 'variation_color_name'})
            list_of_dicts.append(product_info)
        
        elif soup.find_all('span', {'class': 'selection'}):
            product_info['color'] = soup.find_all('span', {'class': 'selection'})
            list_of_dicts.append(product_info)
            
        elif soup.find_all('div', {'class': 'a-row a-spacing-micro'}):
            product_info['color'] = soup.find_all('div', {'class': 'a-row a-spacing-micro'})
            list_of_dicts.append(product_info)
        else:
            product_info['color'] = 'unavailable'
            list_of_dicts.append(product_info)
    
    except:
        product_info['asin'] = i
        
        product_info['name'] = 'unknown product name'

        product_info['description'] = 'unknown details'
        
        product_info['category'] = 'unknown category'
        
        product_info['color'] = 'unknown color' 
        list_of_dicts.append(product_info)
        print(len(list_of_dicts))
    sleep(randint(1,2))
    

print(time.time() - t1)

1565715316.285934
https://www.amazon.com/gp/product/B007WA2TCS
1
https://www.amazon.com/gp/product/B007WA2U7W
2
https://www.amazon.com/gp/product/B007WA2VB2
3
https://www.amazon.com/gp/product/B007WA2ZTA
4
https://www.amazon.com/gp/product/B007WA3558
5
https://www.amazon.com/gp/product/B007WA38IM
6
https://www.amazon.com/gp/product/B007WA38OG
7
https://www.amazon.com/gp/product/B007WA397M
8
https://www.amazon.com/gp/product/B007WA3B5C
9
https://www.amazon.com/gp/product/B007WA3CUQ
10
https://www.amazon.com/gp/product/B007WA3GMK
11
https://www.amazon.com/gp/product/B007WA3HLK
12
https://www.amazon.com/gp/product/B007WA3JE0
13
https://www.amazon.com/gp/product/B007WA3PD0
14
https://www.amazon.com/gp/product/B007WA48E0
15
https://www.amazon.com/gp/product/B007WA49DA
16
https://www.amazon.com/gp/product/B007WA4CQ4
17
https://www.amazon.com/gp/product/B007WACT6Y
18
https://www.amazon.com/gp/product/B007WACX3I
19
https://www.amazon.com/gp/product/B007WACY5K
20
https://www.amazon.com/gp/produ

170
https://www.amazon.com/gp/product/B0085VGQAO
171
https://www.amazon.com/gp/product/B0085XMSG8
172
https://www.amazon.com/gp/product/B0085YC16Y
173
https://www.amazon.com/gp/product/B0085ZSBYO
174
https://www.amazon.com/gp/product/B008607ZA4
175
https://www.amazon.com/gp/product/B00866B69E
176
https://www.amazon.com/gp/product/B00866BCN4
177
https://www.amazon.com/gp/product/B00866BDMY
178
https://www.amazon.com/gp/product/B00866BHO8
179
https://www.amazon.com/gp/product/B00866BIX8
180
https://www.amazon.com/gp/product/B00866BOSW
181
https://www.amazon.com/gp/product/B0086RPNL0
182
https://www.amazon.com/gp/product/B0086RVB1Q
183
https://www.amazon.com/gp/product/B0086ZLY06
184
https://www.amazon.com/gp/product/B0087115JO
185
https://www.amazon.com/gp/product/B008745PZQ
186
https://www.amazon.com/gp/product/B008746WOO
187
https://www.amazon.com/gp/product/B00874MZXQ
188
https://www.amazon.com/gp/product/B00874OFRU
189
https://www.amazon.com/gp/product/B00876TEVK
190
https://www.amaz

KeyboardInterrupt: 

Second scraper

In [26]:
t1 = time.time()
print(t1)
list_of_dicts = []
for i in redo_list2:
    print(f'https://www.amazon.com/gp/product/{i}')
    driver.get(f'https://www.amazon.com/gp/product/{i}')
    soup = BeautifulSoup(driver.page_source, 'lxml') 
    product_info = {}
    try:
        product_info['asin'] = i
        product_info['name'] = soup.find('h1', {'id':'title'}).text.strip()

        product_info['description'] = soup.find('div', {'id':'availability_feature_div'}).text.strip()
        list_of_dicts.append(product_info)
        
    except:
        product_info['asin'] = i
        
        product_info['name'] = 'unknown product name'
        list_of_dicts.append(product_info)
        print(len(list_of_dicts))
    sleep(randint(1,2))
    

print(time.time() - t1)

1565719858.628
https://www.amazon.com/gp/product/B007WA2TCS
https://www.amazon.com/gp/product/B007WA2U7W
2
https://www.amazon.com/gp/product/B007WA2VB2
https://www.amazon.com/gp/product/B007WA2ZTA
https://www.amazon.com/gp/product/B007WA3558
https://www.amazon.com/gp/product/B007WA38IM
6
https://www.amazon.com/gp/product/B007WA38OG
7
https://www.amazon.com/gp/product/B007WA397M
8
https://www.amazon.com/gp/product/B007WA3B5C
9
https://www.amazon.com/gp/product/B007WA3CUQ
10
https://www.amazon.com/gp/product/B007WA3GMK
https://www.amazon.com/gp/product/B007WA3HLK
12
https://www.amazon.com/gp/product/B007WA3JE0
https://www.amazon.com/gp/product/B007WA3PD0
https://www.amazon.com/gp/product/B007WA48E0
https://www.amazon.com/gp/product/B007WA49DA
16
https://www.amazon.com/gp/product/B007WA4CQ4
17
https://www.amazon.com/gp/product/B007WACT6Y
18
https://www.amazon.com/gp/product/B007WACX3I
19
https://www.amazon.com/gp/product/B007WACY5K
https://www.amazon.com/gp/product/B007WAD2QU
21
https://w

https://www.amazon.com/gp/product/B0085ZSBYO
174
https://www.amazon.com/gp/product/B008607ZA4
https://www.amazon.com/gp/product/B00866B69E
176
https://www.amazon.com/gp/product/B00866BCN4
177
https://www.amazon.com/gp/product/B00866BDMY
178
https://www.amazon.com/gp/product/B00866BHO8
179
https://www.amazon.com/gp/product/B00866BIX8
180
https://www.amazon.com/gp/product/B00866BOSW
181
https://www.amazon.com/gp/product/B0086RPNL0
182
https://www.amazon.com/gp/product/B0086RVB1Q
183
https://www.amazon.com/gp/product/B0086ZLY06
https://www.amazon.com/gp/product/B0087115JO
185
https://www.amazon.com/gp/product/B008745PZQ
186
https://www.amazon.com/gp/product/B008746WOO
187
https://www.amazon.com/gp/product/B00874MZXQ
188
https://www.amazon.com/gp/product/B00874OFRU
189
https://www.amazon.com/gp/product/B00876TEVK
190
https://www.amazon.com/gp/product/B00877BFFM
191
https://www.amazon.com/gp/product/B0087AAGQ8
https://www.amazon.com/gp/product/B0087HT1DK
https://www.amazon.com/gp/product/B0

https://www.amazon.com/gp/product/B008EX2ZKI
https://www.amazon.com/gp/product/B008EYO4TW
https://www.amazon.com/gp/product/B008EYOI98
https://www.amazon.com/gp/product/B008EYXW22
https://www.amazon.com/gp/product/B008F05ZP2
https://www.amazon.com/gp/product/B008F06A5Q
https://www.amazon.com/gp/product/B008F06RV8
https://www.amazon.com/gp/product/B008F06T1G
https://www.amazon.com/gp/product/B008F0HXGG
https://www.amazon.com/gp/product/B008F0JKV2
https://www.amazon.com/gp/product/B008F17R2K
https://www.amazon.com/gp/product/B008F4HPLK
https://www.amazon.com/gp/product/B008F4L67O
359
https://www.amazon.com/gp/product/B008F4V5ZM
https://www.amazon.com/gp/product/B008F55KMK
https://www.amazon.com/gp/product/B008F6FYPC
https://www.amazon.com/gp/product/B008F9RCAY
363
https://www.amazon.com/gp/product/B008FC7V7K
https://www.amazon.com/gp/product/B008FC85S4
https://www.amazon.com/gp/product/B008FCBOVE
https://www.amazon.com/gp/product/B008FEMRLS
https://www.amazon.com/gp/product/B008FHHMA6
ht

https://www.amazon.com/gp/product/B008J3EJPW
https://www.amazon.com/gp/product/B008J3OROA
https://www.amazon.com/gp/product/B008J4D1FU
528
https://www.amazon.com/gp/product/B008J4DSU8
https://www.amazon.com/gp/product/B008J4DWTU
https://www.amazon.com/gp/product/B008J4EJC4
https://www.amazon.com/gp/product/B008J4EMOO
https://www.amazon.com/gp/product/B008J4EWWQ
https://www.amazon.com/gp/product/B008J4FJLO
https://www.amazon.com/gp/product/B008J4QOZE
https://www.amazon.com/gp/product/B008J4QT8Q
https://www.amazon.com/gp/product/B008J4RA5M
https://www.amazon.com/gp/product/B008J4S218
https://www.amazon.com/gp/product/B008J4SHOU
https://www.amazon.com/gp/product/B008J52TXE
https://www.amazon.com/gp/product/B008J535MI
https://www.amazon.com/gp/product/B008J53C30
https://www.amazon.com/gp/product/B008J53HOY
https://www.amazon.com/gp/product/B008J5476G
https://www.amazon.com/gp/product/B008J5FR06
https://www.amazon.com/gp/product/B008J5N7V2
https://www.amazon.com/gp/product/B008J7ZUFG
https:

https://www.amazon.com/gp/product/B008MCW402
https://www.amazon.com/gp/product/B008MHFR9C
https://www.amazon.com/gp/product/B008MHRDNA
https://www.amazon.com/gp/product/B008MHZ1RA
https://www.amazon.com/gp/product/B008MKU4TW
https://www.amazon.com/gp/product/B008MLZ5YK
711
https://www.amazon.com/gp/product/B008MMBI5E
https://www.amazon.com/gp/product/B008MMJ27K
https://www.amazon.com/gp/product/B008MMKEU4
https://www.amazon.com/gp/product/B008MNSVMQ
https://www.amazon.com/gp/product/B008MOQ2TO
716
https://www.amazon.com/gp/product/B008MP3UCU
717
https://www.amazon.com/gp/product/B008MP3UOI
718
https://www.amazon.com/gp/product/B008MPH6OI
https://www.amazon.com/gp/product/B008MPV96E
https://www.amazon.com/gp/product/B008MQAOD2
721
https://www.amazon.com/gp/product/B008MRSYLA
https://www.amazon.com/gp/product/B008MVJ9U6
https://www.amazon.com/gp/product/B008MW27TU
https://www.amazon.com/gp/product/B008MWGZFW
https://www.amazon.com/gp/product/B008MWHO8O
https://www.amazon.com/gp/product/B

...

https://www.amazon.com/gp/product/B00EDKKRVA
2950
https://www.amazon.com/gp/product/B00EDKRMGI
https://www.amazon.com/gp/product/B00EDNIY6C
2952
https://www.amazon.com/gp/product/B00EDOR4L2
https://www.amazon.com/gp/product/B00EDQCPT6
2954
https://www.amazon.com/gp/product/B00EDQR316
2955
https://www.amazon.com/gp/product/B00EDRZ9HU
https://www.amazon.com/gp/product/B00EDTKSE2
https://www.amazon.com/gp/product/B00EDTKUAY
https://www.amazon.com/gp/product/B00EDTLGGQ
2959
https://www.amazon.com/gp/product/B00EDTLSMS
2960
https://www.amazon.com/gp/product/B00EDTMCGE
2961
https://www.amazon.com/gp/product/B00EDTMGQK
2962
https://www.amazon.com/gp/product/B00EDTN7TU
2963
https://www.amazon.com/gp/product/B00EDTNPRE
2964
https://www.amazon.com/gp/product/B00EDXZ12M
https://www.amazon.com/gp/product/B00EE1413I
2966
https://www.amazon.com/gp/product/B00EE15C68
https://www.amazon.com/gp/product/B00EE44CQ6
2968
https://www.amazon.com/gp/product/B00EE6VHM6
https://www.amazon.com/gp/product/B00EE7

3123
https://www.amazon.com/gp/product/B00EU3BAS4
3124
https://www.amazon.com/gp/product/B00EU7EKGE
https://www.amazon.com/gp/product/B00EU7OYYW
https://www.amazon.com/gp/product/B00EU865JI
https://www.amazon.com/gp/product/B00EUHTA64
https://www.amazon.com/gp/product/B00EUHUTHS
3129
https://www.amazon.com/gp/product/B00EUIGCS2
3130
https://www.amazon.com/gp/product/B00EUL8RVE
3131
https://www.amazon.com/gp/product/B00EUSCNEY
https://www.amazon.com/gp/product/B00EUVERKY
https://www.amazon.com/gp/product/B00EUXHMVI
https://www.amazon.com/gp/product/B00EV0DM0A
https://www.amazon.com/gp/product/B00EVDMBGI
3136
https://www.amazon.com/gp/product/B00EVDVRBI
3137
https://www.amazon.com/gp/product/B00EVJB0RS
https://www.amazon.com/gp/product/B00EVOHCGQ
https://www.amazon.com/gp/product/B00EVONLSE
https://www.amazon.com/gp/product/B00EVOO9JY
https://www.amazon.com/gp/product/B00EVP2Q5W
https://www.amazon.com/gp/product/B00EVPMBMK
https://www.amazon.com/gp/product/B00EVPPG5Y
https://www.amazon.c

3298
https://www.amazon.com/gp/product/B00FF0CDOQ
3299
https://www.amazon.com/gp/product/B00FF0VCAW
https://www.amazon.com/gp/product/B00FF394VI
https://www.amazon.com/gp/product/B00FF3MPV4
https://www.amazon.com/gp/product/B00FF55Z5U
https://www.amazon.com/gp/product/B00FFIQH5O
https://www.amazon.com/gp/product/B00FFYMUK4
3305
https://www.amazon.com/gp/product/B00FG1WPRE
3306
https://www.amazon.com/gp/product/B00FG1WRHC
https://www.amazon.com/gp/product/B00FG9ZPXM
3308
https://www.amazon.com/gp/product/B00FGCU5M0
https://www.amazon.com/gp/product/B00FGINH6A
https://www.amazon.com/gp/product/B00FGINHUG
https://www.amazon.com/gp/product/B00FGPL53K
3312
https://www.amazon.com/gp/product/B00FGSWUU4
https://www.amazon.com/gp/product/B00FH9I0EM
https://www.amazon.com/gp/product/B00FHL1GUU
https://www.amazon.com/gp/product/B00FHNJHZE
3316
https://www.amazon.com/gp/product/B00FI0LTXE
3317
https://www.amazon.com/gp/product/B00FI0LTXO
https://www.amazon.com/gp/product/B00FI0LUSI
https://www.ama

https://www.amazon.com/gp/product/B00GGKY914
3473
https://www.amazon.com/gp/product/B00GGLFR0A
3474
https://www.amazon.com/gp/product/B00GH0UMXW
https://www.amazon.com/gp/product/B00GHL6R2G
3476
https://www.amazon.com/gp/product/B00GHLMRAW
3477
https://www.amazon.com/gp/product/B00GHQJQF6
https://www.amazon.com/gp/product/B00GHUKZMU
3479
https://www.amazon.com/gp/product/B00GI28574
3480
https://www.amazon.com/gp/product/B00GI6IASO
3481
https://www.amazon.com/gp/product/B00GIIIRZI
https://www.amazon.com/gp/product/B00GIJIKW2
https://www.amazon.com/gp/product/B00GINH3FI
https://www.amazon.com/gp/product/B00GINHC1I
https://www.amazon.com/gp/product/B00GJ0W7FQ
https://www.amazon.com/gp/product/B00GJ9R6Z8
https://www.amazon.com/gp/product/B00GJG91RM
https://www.amazon.com/gp/product/B00GJW29DO
https://www.amazon.com/gp/product/B00GJWE3JW
https://www.amazon.com/gp/product/B00GJXITVE
3491
https://www.amazon.com/gp/product/B00GKC1KC4
https://www.amazon.com/gp/product/B00GKC9P7Q
https://www.ama

3645
https://www.amazon.com/gp/product/B00HTZHXSQ
https://www.amazon.com/gp/product/B00HUOESOS
https://www.amazon.com/gp/product/B00HUOQUM6
https://www.amazon.com/gp/product/B00HUZW7PY
3649
https://www.amazon.com/gp/product/B00HV6O5DY
3650
https://www.amazon.com/gp/product/B00HVEY87E
https://www.amazon.com/gp/product/B00HVXXEOI
3652
https://www.amazon.com/gp/product/B00HWF7KL8
https://www.amazon.com/gp/product/B00HWF7KPY
https://www.amazon.com/gp/product/B00HWF7KR2
https://www.amazon.com/gp/product/B00HWRSQQE
3656
https://www.amazon.com/gp/product/B00HX0CSL4
3657
https://www.amazon.com/gp/product/B00HY2T5TO
3658
https://www.amazon.com/gp/product/B00HYT82E6
https://www.amazon.com/gp/product/B00HZRJLGA
3660
https://www.amazon.com/gp/product/B00I07HQMU
https://www.amazon.com/gp/product/B00I0QK3R6
3662
https://www.amazon.com/gp/product/B00I0XXR5E
3663
https://www.amazon.com/gp/product/B00I16VXD8
https://www.amazon.com/gp/product/B00I19G95W
3665
https://www.amazon.com/gp/product/B00I19J12A


In [27]:
#Check the length of the scrape 
len(list_of_dicts)

3787

In [29]:
#Check the know number of "unknown" rows to make sure it's low 
pd.DataFrame(list_of_dicts)['name'].value_counts()

unknown product name                                                                                                                                                                                     1864
uiphgjwexzv                                                                                                                                                                                                 5
WallFlower Women's Plus Size Luscious Curvy Stretch Bootcut Denim Jeans                                                                                                                                     2
Hanes Ultimate Men's 3-Pack FreshIQ Crew Neck Tee                                                                                                                                                           2
Xiong chao                                                                                                                                                                      

Save first 8000 scrapes to a DataFrame and then to a csv

In [1]:
# product8000 = pd.DataFrame(list_of_dicts)

# product8000.to_csv('product8000.csv')

Saved next 534 scrapes to a DataFrame and then to a csv

In [2]:
# product8534 = pd.DataFrame(list_of_dicts)

# product8534.to_csv('data/product8534')

Saved next 6000 scrapes to a DataFrame and then to a csv

In [29]:
# product14000 = pd.DataFrame(list_of_dicts)

# product14000.to_csv('product14000.csv')

Saved another 6000 scrapes to a DataFrame and then to a csv

In [35]:
# product20000 = pd.DataFrame(list_of_dicts)

# product20000.to_csv('product20000.csv')

Saved next 1180 scrapes to a DataFrame and then to a csv

In [3]:
# product21180 = pd.DataFrame(list_of_dicts)

# product21180.to_csv('product21180.csv')

Saved last section of scrapes to a DataFrame and then to a csv


In [4]:
# product23033 = pd.DataFrame(list_of_dicts)

# product23033.to_csv('product23033.csv')

Many of the above scrapes were blocked by an anti-robot page so i'll need to go through and check for unavailable rows and make a new list of asins to rescrape.

In [30]:
# Function to remove html
def remove_html(text):
    try:
        soup = BeautifulSoup(text, 'lxml')
        html_free = soup.get_text(strip=True)
        return html_free
    except:
        return "missing"

In [31]:
# Function to clean the DataFrame
def EDA(df):
    # Remove html and strip off brackets and 'Color:' from color column
    df['color'] = df['color'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '').str.replace('Color:', '')
    
    # Replace escape characters and white space in Category column 
    df['category'] = df['category'].str.replace('\n', '').str.replace('  ', '')
    
    # Split Category column on the '›' symbol, up to 6 times and return them in a new df where each split is a new column 
    category = df['category'].str.split("›", n=6, expand=True)
    
    # Rename each category split column and add it onto the original df
    df['department'] = category[0]
    df['demographic'] = category[1]
    df['division'] = category[2]
    df['category'] = category[3]
    df['subcategory'] = category[4]
    df['type'] = category[5]
    df['detail_type'] = category[6]
    
    # Remove special characters from the description column 
    df['description'] = df['description'].str.replace('\t', '').str.replace('\n', '')
    
    # Remove html and strip off brackets from details column
    df['details'] = df['details'].apply(lambda x: remove_html(x)).str.replace(']', '').str.replace('[', '')
   
    #Remove html and cut off 'Size:' from the size column 
    df['size'] = df['size'].apply(lambda x: remove_html(x)).str.replace('Size:', '')
    
    return df.head()

In [20]:
product8000 = pd.read_csv('data/product8000.csv')

In [21]:
product20000 = pd.read_csv('data/product20000.csv')

In [22]:
product21180 = pd.read_csv('data/product21180.csv')

In [23]:
product23033 = pd.read_csv('data/product23033.csv')

In [24]:
#A list of DataFrames to be concatonated
dataframes = [product8000, product20000, product21180, product23033]

# Concat all 4 DataFrames into one, along the row axis.
total_product = pd.concat(dataframes, axis=0)

In [25]:
total_product.shape

(17033, 8)

In [26]:
EDA(total_product)

,Unnamed: 0,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,0,0000031887,Dance,unavailable,This fits your . Make sure this fitsby ent...,,Mystiqueshapes Girls Ballet Tutu Neon Lime Green,missing,Sports & Outdoors,Sports & Fitness,Other Sports,Clothing,Girls,Skirts
1,1,0123456479,Jewelry Boxes & Organizers,unavailable,This fits your . Make sure this fitsby ent...,,SHINING IMAGE HUGE PINK LEATHER JEWELRY BOX / ...,missing,"Clothing, Shoes & Jewelry","Shoe, Jewelry & Watch Accessories",Jewelry Accessories,Jewelry Boxes,None,None
2,2,1608299953,None,unavailable,Access for up to 5 family members Download act...,,Learn French: Rosetta Stone French - Level 1,missing,Software,Education & Reference,Languages,None,None,None
3,3,1617160377,None,unavailable,Access for up to 5 family members Download act...,,Learn Italian: Rosetta Stone Italian - Level 1,missing,Software,Education & Reference,Languages,None,None,None
4,4,B00001W0KA,None,Buzz Lightyear,Lead Free Child (4-6 & 7-8) Includes: Bodysuit...,,Buzz Lightyear Boy's Deluxe Toy Story Costume,SelectM(3T-4T)3T-4TSmall (4-6)Medium (7-8)Medi...,Toys & Games,Dress Up & Pretend Play,Costumes,None,None,None


Create a dataframe for only rows that came back unknown.

In [27]:
redo_df = total_product[total_product['department'] == 'unknown category']

In [28]:
redo_df.shape

(12009, 14)

In [29]:
redo_df.isnull().sum()

Unnamed: 0         0
asin               0
category       12009
color              0
description        0
details            0
name               0
size               0
department         0
demographic    12009
division       12009
subcategory    12009
type           12009
detail_type    12009
dtype: int64

In [30]:
redo_list = list(redo_df['asin'])
redo_list

['B00005TQI7',
 'B0000722HK',
 'B00007GD9I',
 'B0000864CZ',
 'B0000866JI',
 'B0000867AN',
 'B0000867GG',
 'B0000867ON',
 'B00008695M',
 'B0000869QI',
 'B000086Q9O',
 'B00008ID0B',
 'B00008ID1O',
 'B00008ID39',
 'B00008IOHR',
 'B00008KI3U',
 'B00008LUS5',
 'B00009QB5V',
 'B0000AT3NT',
 'B0000B35D8',
 'B0000TIIRS',
 'B0000X5EJO',
 'B0000X5FAM',
 'B0000YEC8C',
 'B0000YECC8',
 'B0000ZCH4W',
 'B000147UNU',
 'B00014861K',
 'B00018HRU2',
 'B0001IA06U',
 'B0001VJ2V6',
 'B0001VWSFS',
 'B0001WD5RC',
 'B0001XH7X4',
 'B0001YWBG6',
 'B000200JQI',
 'B00020ZUC6',
 'B00023K3YS',
 'B00029BKAI',
 'B0002DG7ZM',
 'B0002EJ4XS',
 'B0002FHFZQ',
 'B0002FHGBO',
 'B0002FHIM6',
 'B0002FHIMQ',
 'B0002FHIVM',
 'B0002FHIWQ',
 'B0002JA0WC',
 'B0002L01YC',
 'B0002LKLPQ',
 'B0002NZ0LO',
 'B0002TVB70',
 'B0002U71R8',
 'B0002VG2XG',
 'B0002XROVS',
 'B0002ZOQZ8',
 'B00031XFL2',
 'B00061RG1O',
 'B00063J8MW',
 'B00063W3GA',
 'B00064UU1Y',
 'B000662EL6',
 'B00066G516',
 'B000673JT6',
 'B00068VGOU',
 'B0006GK8V4',
 'B0006IXC

In [31]:
len(redo_list)

12009

Save rescrapes to a DataFrame and then all rows that are not unknown to a csv

In [36]:
# redo2000 = pd.DataFrame(list_of_dicts)

#(redo2000[redo2000['department'] != 'unknown category']).to_csv('data/redo1.csv')

In [5]:
#redo6000 = pd.DataFrame(list_of_dicts)

# (redo6000[redo6000['department'] != 'unknown category']).to_csv('data/redo2.csv')

In [6]:
#redo6455 = pd.DataFrame(list_of_dicts)

# (redo6455[redo6455['department'] != 'unknown category']).to_csv('data/redo3.csv')

In [3]:
# Save rescrape to a DataFrame 
# redo8528 = pd.DataFrame(list_of_dicts)

# Save the cleaned data to a csv
# redo8528.to_csv('redo_list1.csv')

# Read in the cleaned data & check shape
redo8528 = pd.read_csv('redo_list1.csv', index_col=[0])
redo8528.shape

(2073, 13)

In [22]:
# Use funtion to clean rescrape 
# EDA(redo8528)

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B007WA2TCS,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
1,B007WA2U7W,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
2,B007WA2VB2,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
3,B007WA2ZTA,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None
4,B007WA3558,None,unknown color,unknown details,missing,unknown product name,missing,unknown category,None,None,None,None,None


In [4]:
#Check value_counts of category column to see if there are any noticable trends.
redo8528['category'].value_counts(dropna=False)

NaN                              1146
 Athletic                          98
 Sandals                           70
 Lingerie, Sleep & Lounge          55
Wrist Watches                      39
 Boots                             35
Flats                              30
Pumps                              30
 Earrings                          29
 Tops, Tees & Blouses              26
 Novelty                           25
 Clothing                          20
Fashion Sneakers                   19
 Jackets & Coats                   18
Jeans                              18
 Scarves & Wraps                   18
 Bracelets                         17
 Dresses                           16
 Running                           15
 Necklaces                         14
 Shirts                            13
 Active                            12
Loafers & Slip-Ons                 12
 Rings                             10
 Underwear                         10
 Sweaters                          10
Sandals     

In [5]:
# check how many product names were collected  
(redo8528[redo8528['name'] != 'unknown product name']).shape

(950, 13)

In [6]:
# Save only rows that have department values to a new csv to be joined on to the clean data in a different notebook.
# (redo8528[redo8528['department'] != 'unknown category']).to_csv('data/redo4.csv')

In [7]:
# Check if number of categories collected matches names
(redo8528[redo8528['department'] != 'unknown category']).shape

(950, 13)

In [8]:
# Saved rescrape 
#redo10000 = pd.DataFrame(list_of_dicts)

In [9]:
# Used function to clean rescraped data
# EDA(redo10000)

In [10]:
# Save known column names to a csv
# (redo10000[redo10000['name'] != 'unknown product name']).to_csv('data/redo5.csv')

In [12]:
# Save clean rescraped data to a csv
#redo10000.to_csv('redo_list2.csv')

redo10000 = pd.read_csv('redo_list2.csv', index_col=[0])
redo10000.head()

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B009L4OGDS,Earrings,unavailable,6mm Crystal Ball Stud Earrings Plain Post and ...,missing,Sterling Silver Pink Crystal Ball Stud Earrings,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Stud,NaN,NaN
1,B009L5081Q,Novelty,unavailable,Size: One size only (small) Total Length: Abou...,missing,Ninimour- Fashion Trendy Women's Stretchy Legg...,missing,"Clothing, Shoes & Jewelry",Novelty & More,Clothing,Women,Leggings,NaN
2,B009L53QXI,Boots,missing,Synthetic Made in USA or Imported Synthetic so...,missing,Dark Tan Faux Leather Bold Red Back Zipper Buc...,missing,"Clothing, Shoes & Jewelry",Women,Shoes,Mid-Calf,NaN,NaN
3,B009L5YU7O,Shoulder Bags,missing,Leathette Imported fabric lining Snap closure ...,missing,MG Collection Yelena Top Handle Soft Hobo Shou...,missing,"Clothing, Shoes & Jewelry",Women,Handbags & Wallets,NaN,NaN,NaN
4,B009LB8CX6,Active,missing,Made in USA or Imported Merino wool blend Fibe...,missing,Kirkland Signature Outdoor Trail Sock Merino W...,missing,"Clothing, Shoes & Jewelry",Women,Clothing,Athletic Socks,NaN,NaN


In [13]:
(redo10000[redo10000['name'] == 'unknown product name'])

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
8,B009LEFPAQ,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
10,B009LEFR4A,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
11,B009LEFS26,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
12,B009LEG2EO,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
13,B009LEG3XO,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
16,B009LM7BWS,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
17,B009LNATJY,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
22,B009LZTK10,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
24,B009M8UF38,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN
28,B009MAW0A2,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN


In [14]:
redo_list2 = (list((redo8528[redo8528['department'] == 'unknown category'])['asin'])) + (list((redo10000[redo10000['department'] == 'unknown category'])['asin']))

len(redo_list2)


2099

In [7]:
redo_list2[:25]

NameError: name 'redo_list2' is not defined

In [17]:
#redo12009 = pd.DataFrame(list_of_dicts)

In [18]:
#EDA(redo12009)

In [60]:
#(redo12009[redo12009['name'] != 'unknown product name']).to_csv('data/redo6.csv')

In [61]:
#redo12009.to_csv('redo_list3.csv')

In [20]:
redo12009 = pd.read_csv('redo_list3.csv', index_col=[0])
redo12009.head()

,asin,category,color,description,details,name,size,department,demographic,division,subcategory,type,detail_type
0,B00B8SDHSU,Loafers & Slip-Ons,missing,100% Leather Imported Rubber sole Shaft measur...,missing,Clarks Men's Portland 2 Easy Slip-On,missing,"Clothing, Shoes & Jewelry",Men,Shoes,NaN,NaN,NaN
1,B00B8TSCUW,Hats & Caps,missing,Women's Light Beret Crochet Beanie Head Circum...,missing,Pop Fashionwear Women's Light Beret Knitted St...,missing,"Clothing, Shoes & Jewelry",Women,Accessories,Berets,NaN,NaN
2,B00B8VFIEI,Men,missing,cotton-blend Premium cotton for a lasting soft...,missing,Nike Premium Cotton Crew Cut Socks (6 Pack) La...,missing,Sports & Outdoors,Sports & Fitness,Clothing,Socks,NaN,NaN
3,B00B8X5X5A,Rings,missing,Width: 7 mm Lightweight and beautiful Packagin...,missing,Chuvora 925 Sterling Silver Woven Celtic Knot ...,missing,"Clothing, Shoes & Jewelry",Women,Jewelry,Bands,NaN,NaN
4,B00B8XGCSW,NaN,unknown color,unknown details,missing,unknown product name,missing,unknown category,NaN,NaN,NaN,NaN,NaN


In [21]:
redo12009['name'].value_counts()

unknown product name                                                                                                                          1688
Elegant Moments Women's Sheer Zebra Print Long Sleeve Bodystocking with Open Crotch                                                              1
Scarleton Two Tone Patent Satchel Bag H1188                                                                                                      1
Pinkyee Women's Floral Bikini Swimwear Cover Up Beach Dress                                                                                      1
Champion Men's Closed Bottom Light Weight Jersey Sweatpant                                                                                       1
Maple Clothing Indian Kurti Women's Tunic Top Printed Blouse India Clothing                                                                      1
INBLUE Men,Women's 2 PCS Stainless Steel Pendant Necklace Jigsaw Puzzle Love Couple Adjustable 20~22 Inch Chain       

In [22]:
redo_list2.extend(list((redo12009[redo12009['name'] == 'unknown product name'])['asin']))


In [23]:
len(redo_list2)

3787

In [33]:
#redo_new = pd.DataFrame(list_of_dicts)

In [42]:
#redo_new.to_csv('redo_list3.csv')

In [44]:
redo_new = pd.read_csv('redo_list3.csv', index_col=[0]).drop(columns='description')
redo_new.head()

,asin,name
0,B007WA2TCS,Allegra K Women V Neck Sleeveless Striped Pane...
1,B007WA2U7W,unknown product name
2,B007WA2VB2,Allegra K Women Scoop Neck Drawstring Mid-Calf...
3,B007WA2ZTA,Allegra K Women Peter Pan 3/4 Sleeve Summer To...
4,B007WA3558,Allegra K Lady Ruffle Tie Neck Short Sleeves B...


In [47]:
redo_new[redo_new['name'] != 'unknown product name'].to_csv('data/redo7.csv')